In [42]:
from datetime import datetime
import requests
import pandas as pd
AJNA_API_URL = 'http://ajna.labin.rf08.srf/ajnaapi/api'

def convert_data_toiso(data):
    try:
        data = datetime.strptime(data, '%d/%m/%Y %H:%M:%S')
    except ValueError:
        data = datetime.strptime(data, '%d/%m/%Y')
    return data.isoformat()


import unicodedata


def ascii_sanitizar(text):
    """Remove marcas de diacríticos (acentos e caracteres especiais).

    Retorna NFC normalizado ASCII
    """
    if not text:
        return None
    return unicodedata.normalize('NFKD', text) \
        .encode('ASCII', 'ignore') \
        .decode('ASCII')

In [43]:
CAMINHO = 'C:/Users/25052288840/Downloads/Planilhas/Junho'


## Planilha Santos Brasil - mapeamento

In [44]:
def processa_df_SBT(filename):
    df = pd.read_excel(filename, engine='odf')
    df['dataevento'] = df['Data hora entrada'].apply(convert_data_toiso)
    return df

In [45]:
mapa_SBT = {'dataevento': ['dtHrOcorrencia', 'dtHrRegistro'],
            'Conteiner': {'listaContainersUld': 'num'},
            'Tipo conteiner': {'listaContainersUld': 'tipo'},
            'Motorista': 'motorista_nome',
            'CPF': 'motorista_cpf',
            'Transportadora': 'nmTransportador',
            'Cnpj': 'cnpjTransportador',
            'Navio': {'listaContainersUld': 'imoNavio'},
            'Porto descarga': {'listaContainersUld': 'portoDescarga'},
            'Porto final': {'listaContainersUld': 'destinoCarga'},
            # Provisório
            'Login': 'login',
            'Mercadoria': 'mercadoria'
}

## Planilha BTP - mapeamento

In [56]:
def processa_planilha_BTP(filename):
    df_BTP = pd.read_excel(filename)
    df_BTP.columns = [ascii_sanitizar(col) for col in df_BTP.columns]
    df_BTP['Entrada Carreta'] = df_BTP['Entrada Carreta'].fillna(method='ffill')
    df_BTP['CNPJ Transportadora'] = df_BTP['CNPJ Transportadora'].fillna(method='ffill')
    df_BTP['Entrada Carreta'] = df_BTP['Entrada Carreta'].astype(str)
    df_BTP['dataevento'] = df_BTP['Entrada Carreta'].apply(convert_data_toiso)
    df_BTP['CNPJ Transportadora'] = df_BTP['CNPJ Transportadora'].apply(lambda x: '{:014.0f}'.format(x))
    df_BTP['Cpf Motorista'] = df_BTP['Cpf Motorista'].apply(lambda x: '{:011.0f}'.format(x))
    df_BTP = df_BTP.fillna('')
    return df_BTP

In [57]:
mapa_BTP = {'dataevento': ['dtHrOcorrencia', 'dtHrRegistro'],
            'Conteiner': {'listaContainersUld': 'num'},
            'Iso Code': {'listaContainersUld': 'tipo'},
            'CNPJ Transportadora': 'cnpjTransportador',
            'Transportadora': 'nmtransportador',
            'Nome Motorista': 'motorista_nome',
            'Cpf Motorista': 'motorista_cpf',
            'Navio Embarque': {'listaContainersUld': 'imoNavio'},
            'Porto Descarga': {'listaContainersUld': 'portoDescarga'},
            'Porto Destino Final': {'listaContainersUld': 'destinoCarga'},
            # Provisório
            'Nome Operador Scanner': 'login',
            'Descricao Ncm': 'mercadoria'
}

## Conexão à API

In [58]:
def get_login_headers():
    rv = requests.post(AJNA_API_URL + '/login',
        json={'username': 'ivan', 'password': 'Ivan1234'})
    # print(rv.text)
    token = rv.json().get('access_token')
    headers = {'Authorization': 'Bearer ' + token}
    return headers




In [59]:
def update_destino(destino, key, valor):
    if isinstance(key, str):
        destino[key] = valor
    elif isinstance(key, list):
        for k in key:
            destino[k] = valor
    elif isinstance(key, dict):
        for subk, subv in key.items():
            subdestino = destino.get(subk)
            if subdestino is None:
                subdestino = {}
                destino[subk] = subdestino
            update_destino(subdestino, subv, valor)

def upload_eventos(recinto: str, mapa: dict, df):
    for index, row in list(df.iterrows()):  # [:10]:
        destino = {'idEvento': hash(recinto + row['dataevento']),
                   'cpfOperOcor': '00000000000',
                   'cpfOperReg': '00000000000',
                   'recinto': recinto,
                   'protocoloEventoRetifCanc': None,
                   'contingencia': False,
                   'codRecintoDestino': 0}
        for key_origem, key_destino in mapa.items():
            update_destino(destino, key_destino, row[key_origem])
        destino['cnpjTransportador'] = destino['cnpjTransportador']\
        .replace('/', '').replace('.', '').replace('-', '')
        destino['listaContainersUld'] = [destino['listaContainersUld']]
        rv = requests.post(AJNA_API_URL + '/acessoveiculo', json=destino, headers=headers)
        if rv.status_code != 201:
            print(destino)
            print(rv.status_code, rv.text)


In [74]:
df_BTP = processa_planilha_BTP(CAMINHO + '/BTP escâner - dia 03.xlsx')
print(len(df_BTP))

2575


In [75]:
headers = get_login_headers()
upload_eventos('BTP', mapa_BTP, df_BTP)

In [76]:
df_SBT = processa_df_SBT(CAMINHO + '/SBT escâner - dia 03.ods')
print(len(df_SBT))

1439


In [77]:
headers = get_login_headers()
upload_eventos('SBT', mapa_SBT, df_SBT)